# Sentiment Analysis of orders reviews in Greek
### Out-of-core-Learning

In [ ]:
!pip show scikit-learn

In [1]:
#Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords
import os

file_path = "Skroutz_dataset.csv"

In [2]:
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0.1,Unnamed: 0,review,label
0,0,0,Πολύ καλό κατάστημα τόσο από κοντά όσο και...,1
1,1,1,Γρήγορη παραλαβή και φιλική εξυπηρέτηση.,1
2,2,2,Εύκολα γρήγορα απλά. Το προϊόν όπως περιγρά...,1
3,3,3,Πολύ ευχαριστημένη με την παράδοση. Επικοινω...,1
4,4,4,Όλα πήγαν καλά με την παραγγελία μου. Θα αγο...,1


In [3]:
df.isna().sum()

Unnamed: 0.1    0
Unnamed: 0      0
review          0
label           0
dtype: int64

In [ ]:
# df.drop(['id'], axis=1, inplace=True)
# df.head()

In [4]:
df.tail()

,Unnamed: 0.1,Unnamed: 0,review,label
6547,6547,6547,Παράγγειλα κινητό 1 έως 3 εργάσιμες ήθελα ...,0
6548,6548,6548,Έχει επιλογές για έκδοση τιμολογίου. Σε αφήν...,0
6549,6549,6549,Η παραγγελία ήταν ένα Huawei p10 lite gold e...,0
6550,6550,6550,Καθυστέρησαν να μου στείλουν την παραγγελία....,0
6551,6551,6551,Δεν ξέρω ακόμη αν θα μου στείλουν το προϊόν!...,0


In [5]:
df.iloc[6547]['review']

'  Παράγγειλα κινητό 1 έως 3 εργάσιμες   ήθελα τιμολόγιο χωρίς ΦΠΑ βάση 39α  με κάλεσαν πίσω να με ενημερώσουν οτι η εταιρία τους δεν κόβουν τιμολόγια χωρίς ΦΠΑ όποτε η παραγγελία θα ακυρωθεί  τους ζήτησα το καταστατικό ώστε να δούμε αν αναγράφεται πουθενά και ακόμη το περιμένω!  Δεν θα τους ξανα προτιμήσω!     '

In [6]:
df.iloc[6547]

Unnamed: 0.1                                                 6547
Unnamed: 0                                                   6547
review            Παράγγειλα κινητό 1 έως 3 εργάσιμες   ήθελα ...
label                                                           0
Name: 6547, dtype: object

In [ ]:
# df['label'].replace({'Negative':0, 'Positive':1}, inplace=True)
# df.head()

In [7]:
# df.to_csv('Skroutz_dataset.csv')

In [7]:
#data = pd.read_csv('/kaggle/working/Skroutz_dataset.csv')
data = pd.read_csv('Skroutz_dataset.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,review,label
0,0,0,Πολύ καλό κατάστημα τόσο από κοντά όσο και...,1
1,1,1,Γρήγορη παραλαβή και φιλική εξυπηρέτηση.,1
2,2,2,Εύκολα γρήγορα απλά. Το προϊόν όπως περιγρά...,1
3,3,3,Πολύ ευχαριστημένη με την παράδοση. Επικοινω...,1
4,4,4,Όλα πήγαν καλά με την παραγγελία μου. Θα αγο...,1


In [8]:
greek_stopwords = stopwords.words('greek')

In [13]:
# remove any html characters
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\\)|\\)|D|P)', text)
    text = re.sub('[\\W]+', ' ', text.lower()) + ' '.join(emotions).replace('-','')
    tokenized = [w for w in text.split() if w not in greek_stopwords]
    return tokenized

In [14]:
# Generate stream_docs
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [15]:
# Testing the function stream_docs
next(stream_docs(path='Skroutz_dataset.csv'))

('0,0,  Πολύ καλό κατάστημα  τόσο από κοντά  όσο και εξ αποστάσεως. Θα ήταν καλό  όμως  όταν λέμε 1-3 εργάσιμες για την παράδοση να διευκρινίζουμε σε εμφανές σημείο ότι δεν μετράνε από την ημέρα της παραγγελίας  αλλά από την ημέρα της τελικής επεξεργασίας της παραγγελίας  οπότε να μην λέμε 1-3 αλλά 2 ως 4 ή οτιδήποτε άλλο ισχύει. Ο πελάτης βλέπει 1 ως 3 και καταλαβαίνει ότι το αργότερο την τρίτη εργάσιμη από την παραγγελία θα λάβει το προϊόν. Δεν είναι υποχρεωμένος να γνωρίζει τις διαδικασίες της εταιρείας  ούτε τον αφορούν. Αν χρειάζεται μια ημέρα (ή 2 ...) για επεξεργασία  ας συνυπολογισθεί στην περιγραφή της παράδοσης του προϊόντος και όχι στα ψιλά γράμματα.     ',
 1)

# Get mini batches

In [16]:
# Get mini batches
def get_minibatch(doc_stream, size):
    docs, y =[], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

# Training the Model

In [17]:
# Training the Model
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [18]:
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)


In [24]:
#clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
clf = SGDClassifier(loss='log_loss', random_state=1, max_iter=1)
#clf = SGDClassifier(loss='hinge', random_state=1, max_iter=1)

In [25]:
doc_stream = stream_docs(path='Skroutz_dataset.csv')

In [26]:
#!pip install pyprind
import pyprind
pbar = pyprind.ProgBar(45)

In [27]:
classes = np.array([0, 1])

In [28]:
progress = ""
for _ in range(6):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break;
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    #pbar.update()
    progress += "#"
    print(progress)

#
##
###
####
#####
######


In [29]:
# Test with the rest 552 data
X_test, y_test = get_minibatch(doc_stream, size=552)
X_test = vect.transform(X_test)

In [30]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.947


In [31]:
clf = clf.partial_fit(X_test, y_test)

In [32]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.998


# Testing the Model

### Example 1

In [33]:
# Test
import pandas as pd
import numpy as np
label = {0: 'negative', 1:'positive'}
example = ['Πολύ καλό κατάστημα  τόσο από κοντά  όσο και εξ αποστάσεως.']
X = vect.transform(example)

In [34]:
# Make a prediction
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: positive
Probability: 63.80%


### Example 2

In [35]:
example = ['Μου αρέσει αυτό το κατάστημα γιατί είναι το χειρότερο από όλα με χάλια εξυπηρέτηση']
X = vect.transform(example)
# Make a prediction
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: negative
Probability: 90.55%


# Save the Model

In [36]:
## Save the Model
import pickle
import os

In [37]:
dest = os.path.join('reviewsclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

In [38]:
pickle.dump(greek_stopwords, open(os.path.join(dest, 'greek_stopwords.pkl'), 'wb'), protocol=4)

In [39]:
pickle.dump(clf, open(os.path.join(dest,'classifier_greek_language.pkl'), 'wb'), protocol=4)

# Test the saved model

In [40]:
# Test the saved model
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle
import numpy as np

In [41]:
files_dir = 'reviewsclassifier'
stop = pickle.load(open(os.path.join(files_dir, 'pkl_objects', 'greek_stopwords.pkl'), 'rb'))

In [43]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\\)|\\)|D|P)', text)
    text = re.sub('[\\W]+', ' ', text.lower()) + ' '.join(emotions).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

# # remove any html characters
# def tokenizer(text):
#     text = re.sub('<[^>]*>', '', text)
#     emotions = re.findall('(?::|;|=)(?:-)?(?:\\)|\\)|D|P)', text)
#     text = re.sub('[\\W]+', ' ', text.lower()) + ' '.join(emotions).replace('-','')
#     tokenized = [w for w in text.split() if w not in greek_stopwords]
#     return tokenized

In [44]:
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)

In [45]:
clf = pickle.load(open(os.path.join(files_dir, 'pkl_objects', 'classifier_greek_language.pkl'), 'rb'))

### Example 1 with Positive review

In [46]:
label = {0:'negative',1:'positive'}
example = ['Πολύ καλή εξυπηρέτηση και γρήγορη παράδοση']
X = vect.transform(example)

In [47]:
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: positive
Probability: 99.02%


### Example 2 with Negative review

In [48]:
label = {0:'negative',1:'positive'}
example = ['το προϊόν δεν ήρθε ποτέ και δεν μας επιστράφηκαν τα χρήματα']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X)) * 100))

Prediction: negative
Probability: 95.51%
